In [8]:
import pandas as pd

## Init bias
Tiene sentido usar el init bias para estudiar el delta bias, si en este caso evaluamos varias veces cambiando las palabras.

Usamos los weats (weat como criterio de optimización) para tener un df con todos los algs

In [9]:
hd = pd.read_csv(r'.\hd\results\bests_hd_weat.csv')
dhd = pd.read_csv(r'.\dhd\results\bests_dhd_weat.csv')
ran = pd.read_csv(r'.\ran\results\bests_ran_weat.csv')
hsr = pd.read_csv(r'.\hsr\results\bests_hsr_weat.csv')


In [10]:
init_bias = pd.read_csv(r'init_bias.csv')

In [11]:
init_metrics =   init_bias.drop(columns=['Unnamed: 0'])   #

In [12]:
init_metrics

,Weat,Weates,RND,RIPA,ECT,RNSB
0,0.246647,0.901104,0.548545,0.190966,0.568189,0.081473
1,0.144591,0.577259,0.211059,0.190374,0.388672,0.104630
2,0.226134,0.848182,0.191214,0.248695,0.565449,0.094981
3,0.177700,0.754676,0.155268,0.224951,0.872865,0.052950
4,0.196311,0.729108,0.217762,0.182797,0.570621,0.105177
5,0.140059,0.861547,0.141913,0.153638,0.718981,0.085191
6,0.169556,0.951932,0.266487,0.200886,0.708492,0.129975
7,0.153538,0.732534,0.243326,0.239662,0.310014,0.158345
8,0.137363,0.680226,0.297200,0.245076,0.675478,0.064678
9,0.155190,0.870039,0.284458,0.156672,0.689642,0.085689


In [13]:
dfs = [hd, dhd, ran, hsr]

In [14]:
for df in dfs:
    df.drop(columns=['Unnamed: 0'], inplace=True)
 

In [15]:
best_aux = []
for df in dfs:
    diff = df.iloc[:, :6] - init_metrics
    last_col = df.iloc[:, -1:] 
    final = pd.concat([diff, last_col], axis=1)
    best_aux.append(final)

In [16]:
names = ['HD', 'DHD', 'RAN', 'HSR']

In [17]:
metrics = ['Weat', 'RND', 'RIPA', 'RNSB', 'ECT', 'Weates']

In [18]:



resultados = {}
prom = []
stds = []
df_res = pd.DataFrame()

for i, df in enumerate(best_aux, start=1):
    promedio = df[metrics].mean()


    desviacion_estandar = df[metrics].std()

    df_resumen = pd.DataFrame({'Promedio': promedio, 'Desviacion_Estandar': desviacion_estandar})
    df_resumen['Algorithm'] = names[i-1]
    df_res = pd.concat([df_res,df_resumen])

In [19]:
df_ttest = df_res.copy()

In [20]:
df_res = df_res.round(3)

In [21]:
df_res['value'] = df_res['Promedio'].astype(str) + ' ± ' + df_res['Desviacion_Estandar'].astype(str)

In [22]:
df_res.drop(columns=['Promedio', 'Desviacion_Estandar'], inplace=True)

In [23]:
df_res

,Algorithm,value
Weat,HD,-0.1 ± 0.031
RND,HD,-0.234 ± 0.102
RIPA,HD,-0.186 ± 0.035
RNSB,HD,-0.073 ± 0.024
ECT,HD,0.14 ± 0.113
Weates,HD,-0.325 ± 0.114
Weat,DHD,-0.033 ± 0.009
RND,DHD,-0.024 ± 0.011
RIPA,DHD,-0.035 ± 0.012
RNSB,DHD,-0.004 ± 0.012


In [24]:
final_df = pd.DataFrame()
for i in names:
    aux = df_res[df_res['Algorithm'] == i].drop(columns=['Algorithm']).transpose().reset_index().rename(columns={'index': 'Algorithm'})
    aux['Algorithm'] = i
    final_df = pd.concat([final_df, aux])

In [25]:
final_df

,Algorithm,Weat,RND,RIPA,RNSB,ECT,Weates
0,HD,-0.1 ± 0.031,-0.234 ± 0.102,-0.186 ± 0.035,-0.073 ± 0.024,0.14 ± 0.113,-0.325 ± 0.114
0,DHD,-0.033 ± 0.009,-0.024 ± 0.011,-0.035 ± 0.012,-0.004 ± 0.012,0.008 ± 0.026,-0.056 ± 0.036
0,RAN,-0.099 ± 0.037,-0.233 ± 0.101,-0.19 ± 0.036,-0.074 ± 0.034,0.15 ± 0.123,-0.341 ± 0.129
0,HSR,-0.124 ± 0.032,-0.022 ± 0.092,-0.157 ± 0.036,-0.042 ± 0.033,-0.151 ± 0.082,-0.229 ± 0.082


## ttest 

In [30]:
import numpy as np
from scipy.stats import t

In [31]:
def corrected_ttest(baslein,alg):
    differences = baslein.values - alg.values
    mean_diff = np.mean(differences)
    variance_diff = np.var(differences, ddof=1)

    k = 10  # Número de particiones (folds)
    n = len(baslein)  # Tamaño de la muestra
    corrected_t_stat = mean_diff / np.sqrt((variance_diff / n) * (1 + 1/k))
    df = n - 1

    # Calcular el valor p
    p_value = 2 * (1 - t.cdf(np.abs(corrected_t_stat), df))

    return p_value

In [32]:
df_alls = pd.DataFrame()
for i in range(4):
    df = pd.melt(best_aux[i].iloc[:, :-1], var_name='Metric', value_name='Value')
    df['Algorithm'] = names[i]
    df_alls = pd.concat([df_alls, df])

In [33]:
from scipy.stats import ttest_ind

dhd = {}
hsr = {}
ran = {}


for i in metrics:
    
    basline = df_alls[(df_alls['Algorithm'] == 'HD') & (df_alls['Metric'] == i)]['Value']
    dhd[i] = corrected_ttest(basline, df_alls[(df_alls['Algorithm'] == 'DHD') & (df_alls['Metric'] == i)]['Value']) 
    hsr[i] = corrected_ttest(basline, df_alls[(df_alls['Algorithm'] == 'HSR') & (df_alls['Metric'] == i)]['Value'])
    ran[i] = corrected_ttest(basline, df_alls[(df_alls['Algorithm'] == 'RAN') & (df_alls['Metric'] == i)]['Value']) 
  


In [34]:
p_values = pd.DataFrame([hsr,dhd,ran],index=[0,1,2])

In [35]:
p_values['Algorithm'] = ['HSR', 'DHD', 'RAN']

In [36]:
p_values

,Weat,RND,RIPA,RNSB,ECT,Weates,Algorithm
0,0.008111,0.004825,1.269450e-05,0.000525,0.000019,0.076222,HSR
1,0.000019,0.000300,2.005101e-07,0.000002,0.008334,0.000022,DHD
2,0.934315,0.825253,1.714813e-02,0.902839,0.674645,0.562755,RAN


In [37]:
p_values[p_values.drop(columns=['Algorithm']) < 0.05]

,Weat,RND,RIPA,RNSB,ECT,Weates,Algorithm
0,0.008111,0.004825,1.269450e-05,0.000525,0.000019,NaN,NaN
1,0.000019,0.000300,2.005101e-07,0.000002,0.008334,0.000022,NaN
2,NaN,NaN,1.714813e-02,NaN,NaN,NaN,NaN


## Violin plot

### read data

In [39]:
hd_weat = pd.read_csv(r'.\hd\results\bests_hd_weat.csv')
dhd_weat = pd.read_csv(r'.\dhd\results\bests_dhd_weat.csv')
ran_weat = pd.read_csv(r'.\ran\results\bests_ran_weat.csv')
hsr_weat = pd.read_csv(r'.\hsr\results\bests_hsr_weat.csv')

In [40]:
hd_ripa = pd.read_csv(r'.\hd\results\bests_hd_ripa.csv')
dhd_ripa = pd.read_csv(r'.\dhd\results\bests_dhd_ripa.csv')
ran_ripa = pd.read_csv(r'.\ran\results\bests_ran_ripa.csv')
hsr_ripa = pd.read_csv(r'.\hsr\results\bests_hsr_ripa.csv')

In [41]:
hd_ect = pd.read_csv(r'.\hd\results\bests_hd_ect.csv')
dhd_ect = pd.read_csv(r'.\dhd\results\bests_dhd_ect.csv')
ran_ect = pd.read_csv(r'.\ran\results\bests_ran_ect.csv')
hsr_ect = pd.read_csv(r'.\hsr\results\bests_hsr_ect.csv')

In [42]:
hd_rnd = pd.read_csv(r'.\hd\results\bests_hd_rnd.csv')
dhd_rnd = pd.read_csv(r'.\dhd\results\bests_dhd_rnd.csv')
ran_rnd = pd.read_csv(r'.\ran\results\bests_ran_rnd.csv')
hsr_rnd = pd.read_csv(r'.\hsr\results\bests_hsr_rnd.csv')

In [43]:
hd_rnsb = pd.read_csv(r'.\hd\results\bests_hd_rnsb.csv')
dhd_rnsb = pd.read_csv(r'.\dhd\results\bests_dhd_rnsb.csv')
ran_rnsb = pd.read_csv(r'.\ran\results\bests_ran_rnsb.csv')
hsr_rnsb = pd.read_csv(r'.\hsr\results\bests_hsr_rnsb.csv')


In [44]:
hd_weates = pd.read_csv(r'.\hd\results\bests_hd_weates.csv')
dhd_weates = pd.read_csv(r'.\dhd\results\bests_dhd_weates.csv')
ran_weates = pd.read_csv(r'.\ran\results\bests_ran_weates.csv')
hsr_weates = pd.read_csv(r'.\hsr\results\bests_hsr_weates.csv')

##  bests

In [49]:
# juntar todos los dataframes hd
hd = pd.concat([hd_weat, hd_ripa, hd_ect, hd_rnd, hd_rnsb, hd_weates], axis=0)
hd.drop(columns=['Unnamed: 0'], inplace=True)
hd_melt = pd.melt(hd, var_name='Metric', value_name='Value')
hd_melt['Algorithm'] = 'HD'

#juntar todos los dataframes dhd
dhd = pd.concat([dhd_weat, dhd_ripa, dhd_ect, dhd_rnd, dhd_rnsb, dhd_weates], axis=0)
dhd.drop(columns=['Unnamed: 0'], inplace=True)
dhd_melt = pd.melt(dhd, var_name='Metric', value_name='Value')
dhd_melt['Algorithm'] = 'DHD'

#juntar todos los dataframes ran
ran = pd.concat([ran_weat, ran_ripa, ran_ect, ran_rnd, ran_rnsb, ran_weates], axis=0)
ran.drop(columns=['Unnamed: 0'], inplace=True)
ran_melt = pd.melt(ran, var_name='Metric', value_name='Value')
ran_melt['Algorithm'] = 'RAN'

#juntar todos los dataframes hsr
hsr = pd.concat([hsr_weat, hsr_ripa, hsr_ect, hsr_rnd, hsr_rnsb, hsr_weates], axis=0)
hsr.drop(columns=['Unnamed: 0'], inplace=True)
#hsr_melt = pd.melt(hsr, var_name='Metric', value_name='Value')
#hsr_melt['Algorithm'] = 'HSR'

In [50]:
dhd.groupby(['solver',]).count()

,Weat,Weates,RND,RIPA,ECT,RNSB
solver,,,,,,
arpack,18,18,18,18,18,18
auto,21,21,21,21,21,21
full,4,4,4,4,4,4
randomized,17,17,17,17,17,17


In [51]:
hd.groupby('solver').count()

,Weat,Weates,RND,RIPA,ECT,RNSB
solver,,,,,,
arpack,19,19,19,19,19,19
auto,23,23,23,23,23,23
full,3,3,3,3,3,3
randomized,15,15,15,15,15,15


In [52]:
hsr.groupby('alpha').count()

,Weat,Weates,RND,RIPA,ECT,RNSB
alpha,,,,,,
0,45,45,45,45,45,45
10,7,7,7,7,7,7
60,1,1,1,1,1,1
80,2,2,2,2,2,2
100,5,5,5,5,5,5


In [53]:
ran.groupby('[theta,neighbour]').count()

,Weat,Weates,RND,RIPA,RNSB,ECT
"[theta,neighbour]",,,,,,
"[0.02, 100]",10,10,10,10,10,10
"[0.02, 200]",18,18,18,18,18,18
"[0.02, 50]",6,6,6,6,6,6
"[0.05, 100]",3,3,3,3,3,3
"[0.05, 200]",3,3,3,3,3,3
"[0.05, 50]",4,4,4,4,4,4
"[0.1, 100]",2,2,2,2,2,2
"[0.1, 200]",4,4,4,4,4,4
"[0.1, 50]",10,10,10,10,10,10


### Plot

In [54]:

hd = pd.concat([hd_weat, hd_ripa, hd_ect, hd_rnd, hd_rnsb, hd_weates], axis=0)

In [55]:
hd.drop(columns=['Unnamed: 0','solver'], inplace=True)
hd_melt = pd.melt(hd, var_name='Metric', value_name='Value')


In [56]:
hd_melt['Algorithm'] = 'HD'

In [58]:
#juntar todos los dataframes dhd
dhd = pd.concat([dhd_weat, dhd_ripa, dhd_ect, dhd_rnd, dhd_rnsb, dhd_weates], axis=0)
dhd.drop(columns=['Unnamed: 0','solver'], inplace=True)
dhd_melt = pd.melt(dhd, var_name='Metric', value_name='Value')
dhd_melt['Algorithm'] = 'DHD'

In [59]:
#juntar todos los dataframes ran
ran = pd.concat([ran_weat, ran_ripa, ran_ect, ran_rnd, ran_rnsb, ran_weates], axis=0)
ran.drop(columns=['Unnamed: 0','[theta,neighbour]'], inplace=True)
ran_melt = pd.melt(ran, var_name='Metric', value_name='Value')
ran_melt['Algorithm'] = 'RAN'

In [60]:
#juntar todos los dataframes hsr
hsr = pd.concat([hsr_weat, hsr_ripa, hsr_ect, hsr_rnd, hsr_rnsb, hsr_weates], axis=0)
hsr.drop(columns=['Unnamed: 0','alpha'], inplace=True)
hsr_melt = pd.melt(hsr, var_name='Metric', value_name='Value')
hsr_melt['Algorithm'] = 'HSR'

In [61]:

all_results = pd.concat([hd_melt, dhd_melt, ran_melt, hsr_melt], axis=0)

In [62]:
weats = all_results[all_results['Metric'] == 'Weat']

In [63]:
ects = all_results[all_results['Metric'] == 'ECT']

In [64]:
rnsbs = all_results[all_results['Metric'] == 'RNSB']

In [65]:
rnds = all_results[all_results['Metric'] == 'RND']

In [66]:
weatess = all_results[all_results['Metric'] == 'Weates']

In [67]:
ripas = all_results[all_results['Metric'] == 'RIPA']

In [68]:
import plotly.graph_objects as go

import pandas as pd

#df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/violin_data.csv")

fig = go.Figure()

#days = ['Thur', 'Fri', 'Sat', 'Sun']

for met in names:
    fig.add_trace(go.Violin(x=weats['Algorithm'][weats['Algorithm'] == met],
                            y=weats['Value'][weats['Algorithm'] == met],
                            name=met,
                            box_visible=True,
                            meanline_visible=True))
    
fig.update_layout(title="WEAT")

fig.show()

In [69]:
import plotly.graph_objects as go

import pandas as pd

#df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/violin_data.csv")

fig = go.Figure()

#days = ['Thur', 'Fri', 'Sat', 'Sun']

for met in names:
    fig.add_trace(go.Violin(x=ects['Algorithm'][ects['Algorithm'] == met],
                            y=ects['Value'][ects['Algorithm'] == met],
                            name=met,
                            box_visible=True,
                            meanline_visible=True))

fig.show()

In [70]:
import plotly.graph_objects as go

import pandas as pd

#df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/violin_data.csv")

fig = go.Figure()

#days = ['Thur', 'Fri', 'Sat', 'Sun']

for met in names:
    fig.add_trace(go.Violin(x=rnsbs['Algorithm'][rnsbs['Algorithm'] == met],
                            y=rnsbs['Value'][rnsbs['Algorithm'] == met],
                            name=met,
                            box_visible=True,
                            meanline_visible=True))

fig.show()

In [71]:
import plotly.graph_objects as go

import pandas as pd

#df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/violin_data.csv")

fig = go.Figure()

#days = ['Thur', 'Fri', 'Sat', 'Sun']

for met in names:
    fig.add_trace(go.Violin(x=weatess['Algorithm'][weatess['Algorithm'] == met],
                            y=weatess['Value'][weatess['Algorithm'] == met],
                            name=met,
                            box_visible=True,
                            meanline_visible=True))

fig.show()

In [72]:
import plotly.graph_objects as go

import pandas as pd

#df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/violin_data.csv")

fig = go.Figure()

#days = ['Thur', 'Fri', 'Sat', 'Sun']

for met in names:
    fig.add_trace(go.Violin(x=ripas['Algorithm'][ripas['Algorithm'] == met],
                            y=ripas['Value'][ripas['Algorithm'] == met],
                            name=met,
                            box_visible=True,
                            meanline_visible=True))

fig.show()